In [1]:
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from transformers import AutoModelForCausalLM, AutoTokenizer

# 加载模型和 tokenizer
model_path = "./Qwen2.5-0.5B-Instruct-jinm"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

def get_bio_tags(text):
    """
    调用大模型进行 BIO 标签预测
    """
    sys_prompt = """
    你是一个医疗命名实体专家！请根据当前对话文本内容，识别出每一句话中的BIO实体标签
    """
    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": text}
    ]

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    generated_ids = model.generate(**model_inputs, max_new_tokens=512)
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

def calculate_f1(y_true, y_pred):
    """
    计算 F1 分数
    """
    true_labels = [label.split() for label in y_true]
    pred_labels = [label.split() for label in y_pred]

    true_labels_flat = [item for sublist in true_labels for item in sublist]
    print("true_labels_flat:", true_labels_flat)
    pred_labels_flat = [item for sublist in pred_labels for item in sublist]
    print("pred_labels_flat", pred_labels_flat)

    precision = precision_score(true_labels_flat, pred_labels_flat, average='micro')
    recall = recall_score(true_labels_flat, pred_labels_flat, average='micro')
    f1 = f1_score(true_labels_flat, pred_labels_flat, average='micro')
    accuracy = accuracy_score(true_labels_flat, pred_labels_flat)
    return precision, recall, f1

# 读取 TSV 文件
data = pd.read_csv("dev.tsv", sep='\t', encoding='utf8')

# 随机抽取100条数据
sampled_data = data.sample(n=100, random_state=42)

results = []
predicted_labels = []
true_labels = list(sampled_data["output"])

# 创建列表来存储预测内容
predictions = []
problematic = 0

for idx, row in tqdm(sampled_data.iterrows(), total=len(sampled_data)):
    instruction = row["instruction"]
    actual_output = row["output"]

    # 获取预测输出
    predicted_output = get_bio_tags(instruction).strip()  # 去掉首尾空白字符

    # 检查预测输出和实际输出的位置和标签是否一致，仅需对比
    actual_tags = actual_output.split()
    print("actual_tags",actual_tags)
    predicted_tags = predicted_output.split()
    print("predicted_tags",predicted_tags)

    if len(predicted_tags) != len(actual_tags) or predicted_tags != actual_tags:
        print(f"预测格式错误: index={idx}, instruction={instruction}, predicted_output={predicted_output}")
        # predicted_output = "O " * len(actual_tags)  # 使用默认值避免程序中断
        predicted_output = "X " * len(actual_tags)  # 使用默认值避免程序中断
        predicted_output = predicted_output.strip()  # 去掉最后的多余空格
        problematic += 1

    # 将 predicted_output 添加到 predicted_labels 列表中
    predicted_labels.append(predicted_output)
    predictions.append([instruction, actual_output, predicted_output])
    results.append(predicted_output == actual_output)

print(f"输出格式问题的条目个数： {problematic}")

  1%|          | 1/100 [00:06<10:49,  6.56s/it]

actual_tags ['O', 'O', 'O', 'B-Drug', 'I-Drug', 'I-Drug']
predicted_tags ['O', 'O', 'O', 'B-Drug', 'I-Drug', 'I-Drug']


  2%|▏         | 2/100 [00:10<08:14,  5.04s/it]

actual_tags ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
predicted_tags ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


  3%|▎         | 3/100 [00:17<09:44,  6.03s/it]

...

actual_tags ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
predicted_tags ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


 99%|█████████▉| 99/100 [09:34<00:04,  4.92s/it]

actual_tags ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
predicted_tags ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


100%|██████████| 100/100 [09:35<00:00,  5.76s/it]

actual_tags ['O', 'O', 'O', 'O', 'O']
predicted_tags ['O', 'O', 'O', 'O', 'O']
输出格式问题的条目个数： 14


In [2]:
# 计算 F1 分数
precision, recall, f1 = calculate_f1(true_labels, predicted_labels)

print(f"精确率(Precision): {precision}")
print(f"召回率(Recall): {recall}")
print(f"F1 分数(F1 Score): {f1}")


# 打印预测内容
print(predictions)

# 将预测内容保存为新的 TSV 文件
output_df = pd.DataFrame(predictions, columns=['instruction', 'output', 'pred_output'])
output_df.to_csv("predictions_sampled.tsv", sep='\t', index=False)

print("随机抽样预测结果已保存到 predictions_sampled1.tsv 文件中")

true_labels_flat: ['O', 'O', 'O', 'B-Drug', 'I-Drug', 'I-Drug', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Symptom', 'I-Symptom', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Operation', 'I-Operation', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Symptom', 'I-Symptom', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Symptom', 'I-Symptom', 'O', 'O', 'O', 'O', 'B-Drug_Category', 'I-Drug_Category', 'I-Drug_Category', 'I-Drug_Category', 'I-Drug_Category', 'I-Drug_Category', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Drug', 'I-Drug', 'I-Drug', 'I-Drug', 'O', 'B-Drug', 'I-Drug', 'I-Drug', 'I-Drug', 'I-Drug', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Drug', 'I-Drug', 'I-Drug', 'I-Drug', 'B-Drug', 'I-Drug', 'I-Drug', 'I-Drug', 'I-Drug', 'I-Drug',